In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#장고에서는 어떻게 처리가능한 부분인지 검토 필요!
bertmodel, vocab = get_pytorch_kobert_model()
device = torch.device("cpu")

In [ ]:
#파이토치 클래스 정의
class BERTClassifier1(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=1, dr_rate=None, params=None):
        super(BERTClassifier1, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier1 = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout1 = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout1(pooler)
        return self.classifier1(out)


In [ ]:
class BERTClassifier2(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=None, params=None):
        super(BERTClassifier2, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier2 = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout2 = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout2(pooler)
        return self.classifier2(out)

In [ ]:
class BERTClassifier3(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=None, params=None):
        super(BERTClassifier3, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier3 = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout3 = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout3(pooler)
        return self.classifier3(out)

In [ ]:
class BERTClassifier4(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=10, dr_rate=None, params=None):
        super(BERTClassifier4, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier4 = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout4 = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout4(pooler)
        return self.classifier4(out)

In [ ]:
#학습모델 불러오기.
model1 = BERTClassifier1(bertmodel, dr_rate=0.5).to(device)
model2 = BERTClassifier2(bertmodel, dr_rate=0.5).to(device)
model3 = BERTClassifier3(bertmodel, dr_rate=0.5).to(device)
model4 = BERTClassifier4(bertmodel, dr_rate=0.5).to(device)

checkpoint=torch.load('wbs_classifier_ver101.pth') #pth 파일 경로 설정
model1.load_state_dict(checkpoint['model1_state_dict'])
model2.load_state_dict(checkpoint['model2_state_dict'])
model3.load_state_dict(checkpoint['model3_state_dict'])
model4.load_state_dict(checkpoint['model4_state_dict'])

model1.eval()
model2.eval()
model3.eval()
model4.eval()

In [ ]:
#데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#parameter 정의
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
max_len = 64              
batch_size = 32

In [ ]:
#예측 모델 함수 정의
def predictlv3(predict_sentence):
    
    data1 = [predict_sentence, '0']
    dataset_another1 = [data1]

    another_test1 = BERTDataset(dataset_another1, 0, 1, tok, max_len, True, False)
    test_dataloader1 = torch.utils.data.DataLoader(another_test1, batch_size=batch_size, num_workers=5)

    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader1):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)
        out = model1(token_ids, valid_length, segment_ids)


        total=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                total.append("토공") #학습된 WBS 내용에 따라 목록 추가
        
                
        return total[0]

In [ ]:
def predictlv4(predict_sentence):
    
    data2 = [predict_sentence, '0']
    dataset_another2 = [data2]

    another_test2 = BERTDataset(dataset_another2, 0, 1, tok, max_len, True, False)
    test_dataloader2 = torch.utils.data.DataLoader(another_test2, batch_size=batch_size, num_workers=5)

    model2.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader2):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)


        total=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0: #학습된 WBS 내용에 따라 목록 추가
                total.append("본선")
            elif np.argmax(logits) == 1:
                total.append("IC/JC")
            elif np.argmax(logits) == 2:
                total.append("지선/부체도로")
            
                       
        return total[0]

In [ ]:
def predictlv7_1(predict_sentence):
    
    data3 = [predict_sentence, '0']
    dataset_another3 = [data3]

    another_test3 = BERTDataset(dataset_another3, 0, 1, tok, max_len, True, False)
    test_dataloader3 = torch.utils.data.DataLoader(another_test3, batch_size=batch_size, num_workers=5)

    model3.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader3):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)
        out = model3(token_ids, valid_length, segment_ids)


        total=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0: #학습된 WBS 내용에 따라 목록 추가
                total.append("흙깍기")
            elif np.argmax(logits) == 1:
                total.append("흙쌓기")
            elif np.argmax(logits) == 2:
                total.append("토공기타")
               
            
            
                
        return total[0]

In [ ]:
def predictlv7_2(predict_sentence):
    
    data4 = [predict_sentence, '0']
    dataset_another4 = [data4]

    another_test4 = BERTDataset(dataset_another4, 0, 1, tok, max_len, True, False)
    test_dataloader4 = torch.utils.data.DataLoader(another_test4, batch_size=batch_size, num_workers=5)

    model4.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader4):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)
        out = model4(token_ids, valid_length, segment_ids)


        total=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0: #학습된 WBS 내용에 따라 목록 추가
                total.append("노상")
            elif np.argmax(logits) == 1:
                total.append("노체")
            elif np.argmax(logits) == 2:
                total.append("리핑")
            elif np.argmax(logits) == 3:
                total.append("발파")
            elif np.argmax(logits) == 4:
                total.append("비탈면보호공")
            elif np.argmax(logits) == 5:
                total.append("연약지반처리")
            elif np.argmax(logits) == 6:
                total.append("옹벽기타")
            elif np.argmax(logits) == 7:
                total.append("토공기타")
            elif np.argmax(logits) == 8:
                total.append("흙깍기기타")
            elif np.argmax(logits) == 9:
                total.append("흙쌓기기타")     
           
            
                
        return total[0]

In [ ]:
#사전 정의된 분석 문서 불러오기
input_data = ['RAMP-E 0+660 ~ 0+720 암 성토(B)', 'RAMP-E 0+600 ~ 0+800 2공구 반입토 운반 및 성토(C)', '부체도로 35;2 : 임목폐기물 집목[C]']
#웹서비스에서 각각 입력된 작업일지의 내용을 가져옴. ex. ['text1', 'text2', 'text3']
#코드 수정 필요.

In [ ]:
#예측 모델 적용 함수 정의 (WBS단계별로 정의)
def execute_prediction(values, p3, p4, pa7_1, pb7_1):
    p3.append(predictlv3(values))
    p4.append(predictlv4(values))
    pa7_1.append(predictlv7_1(values))
    pb7_1.append(predictlv7_2(values))

In [34]:

#구간별(각행)에 대하여 예측 모델 적용.
df = pd.DataFrame(columns=['I3', 'I4', 'I5', 'I7'])
#df는 공종, 시설물, 부위, 작업자관리단위를 컬럼명으로 가지고 각각의 입력값의 번호를 행번호로 가지는 데이터프레임
for i, cvalues in enumerate(input_data):
    p3, p4, pa7_1, pb7_1 = [], [], [], [] #WBS 단계별 리스트 정의
    execute_prediction(cvalues, p3, p4, pa7_1, pb7_1) #예측 모델 적용
    row_data = {'I3': p3[0], 'I4': p4[0], 'I5': pa7_1[0], 'I7': pb7_1[0]}
    df = df.append(row_data, ignore_index=True)
#이후 데이터프레임에서 DB 정보와 비교하여 일치하는 내용을 추출하면 됨. 23.03.22 기준 웹서비스에서는
#I5와 I7만 내용만 작성하도록 되어 있어 해당 열의 정보만 비교하면 될 것으로 판단됨.
